# Student Performance Prediction Models

This notebook develops and validates predictive models for Bangladesh student educational outcomes:
- Academic performance prediction
- Dropout risk identification
- Grade progression modeling
- Intervention effectiveness prediction
- Feature importance analysis
- Model interpretability and validation

**Machine Learning Models Include:**
- Linear and polynomial regression
- Random Forest and Gradient Boosting
- Support Vector Machines
- Neural Networks
- Ensemble methods
- Model evaluation and selection

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys
from pathlib import Path

# Machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
import joblib

# Add project root to Python path
sys.path.append('../..')
from src.data_processing.data_processor import DataProcessor

# Configure display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')

# Set up plotting
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette('viridis')

## 1. Data Preparation and Feature Engineering

In [ ]:
# Create comprehensive dataset for modeling
np.random.seed(42)
n_students = 5000

# Generate synthetic student data with realistic relationships
modeling_data = pd.DataFrame({
    'student_id': [f'S{i:04d}' for i in range(1, n_students + 1)],
    'division': np.random.choice(['Dhaka', 'Chittagong', 'Khulna', 'Rajshahi', 'Sylhet', 'Barishal', 'Rangpur', 'Mymensingh'], n_students),
    'gender': np.random.choice(['Male', 'Female'], n_students),
    'age': np.random.randint(13, 19, n_students),
    'grade_level': np.random.choice([6, 7, 8, 9, 10, 11, 12], n_students),
    'institution_type': np.random.choice(['Government', 'Private', 'Madrasa'], n_students, p=[0.65, 0.25, 0.10]),
    'socioeconomic_status': np.random.choice(['Low', 'Medium', 'High'], n_students, p=[0.45, 0.35, 0.20]),
    'area_type': np.random.choice(['Urban', 'Rural'], n_students, p=[0.35, 0.65]),
    'mother_education': np.random.choice(['No Education', 'Primary', 'Secondary', 'Higher Secondary', 'University'], n_students, p=[0.3, 0.25, 0.25, 0.15, 0.05]),
    'father_education': np.random.choice(['No Education', 'Primary', 'Secondary', 'Higher Secondary', 'University'], n_students, p=[0.25, 0.25, 0.25, 0.15, 0.10]),
    'family_income': np.random.choice(['Very Low', 'Low', 'Medium', 'High', 'Very High'], n_students, p=[0.25, 0.30, 0.25, 0.15, 0.05]),
    'previous_year_gpa': np.random.uniform(1.5, 4.5, n_students),
    'days_absent_last_term': np.random.poisson(5, n_students),
    'extracurricular_activities': np.random.choice([0, 1, 2, 3], n_students, p=[0.4, 0.3, 0.2, 0.1]),
    'home_internet': np.random.choice([0, 1], n_students, p=[0.6, 0.4]),
    'study_hours_per_day': np.random.uniform(1, 8, n_students),
    'teacher_student_ratio': np.random.uniform(25, 60, n_students),
    'school_infrastructure_score': np.random.uniform(2, 10, n_students)
})

# Create target variable with realistic relationships
# Base GPA influenced by multiple factors
base_gpa = 2.5 + (modeling_data['previous_year_gpa'] - 2.5) * 0.7  # Strong correlation with previous GPA

# Add effects of various factors
ses_effect = modeling_data['socioeconomic_status'].map({'Low': -0.3, 'Medium': 0, 'High': 0.4})
area_effect = modeling_data['area_type'].map({'Rural': -0.2, 'Urban': 0.2})
inst_effect = modeling_data['institution_type'].map({'Government': 0, 'Private': 0.3, 'Madrasa': -0.1})
gender_effect = modeling_data['gender'].map({'Male': 0, 'Female': 0.1})

# Education effects
edu_mapping = {'No Education': 0, 'Primary': 0.1, 'Secondary': 0.2, 'Higher Secondary': 0.3, 'University': 0.4}
mother_edu_effect = modeling_data['mother_education'].map(edu_mapping) * 0.2
father_edu_effect = modeling_data['father_education'].map(edu_mapping) * 0.15

# Other effects
attendance_effect = -modeling_data['days_absent_last_term'] * 0.02
study_effect = modeling_data['study_hours_per_day'] * 0.05
extracurricular_effect = modeling_data['extracurricular_activities'] * 0.1
internet_effect = modeling_data['home_internet'] * 0.15
infrastructure_effect = (modeling_data['school_infrastructure_score'] - 6) * 0.05

# Combine all effects with some noise
noise = np.random.normal(0, 0.3, n_students)

modeling_data['current_gpa'] = (
    base_gpa + ses_effect + area_effect + inst_effect + gender_effect +
    mother_edu_effect + father_edu_effect + attendance_effect + 
    study_effect + extracurricular_effect + internet_effect + 
    infrastructure_effect + noise
).clip(0, 5)

# Create dropout risk indicator
dropout_prob = (
    0.05 +  # Base dropout rate
    (modeling_data['days_absent_last_term'] > 15) * 0.3 +  # High absenteeism
    (modeling_data['current_gpa'] < 2.0) * 0.4 +  # Low performance
    (modeling_data['socioeconomic_status'] == 'Low') * 0.2 +  # Economic factors
    (modeling_data['area_type'] == 'Rural') * 0.1  # Geographic factors
)

modeling_data['dropout_risk'] = np.random.binomial(1, dropout_prob.clip(0, 1), n_students)

print(f"Dataset created: {modeling_data.shape}")
print(f"Target variable (current_gpa) statistics:")
print(modeling_data['current_gpa'].describe())
print(f"\nDropout risk rate: {modeling_data['dropout_risk'].mean():.2%}")

modeling_data.head()

## 2. Feature Engineering and Preprocessing

In [ ]:
# Feature engineering
def create_features(df):
    """Create additional features for modeling."""
    df_features = df.copy()
    
    # Numerical features
    df_features['age_grade_ratio'] = df_features['age'] / df_features['grade_level']
    df_features['gpa_improvement_potential'] = 5.0 - df_features['previous_year_gpa']
    df_features['attendance_rate'] = 1 - (df_features['days_absent_last_term'] / 200)  # Assuming 200 school days
    df_features['study_intensity'] = df_features['study_hours_per_day'] / df_features['age']
    
    # Categorical feature combinations
    df_features['ses_area'] = df_features['socioeconomic_status'] + '_' + df_features['area_type']
    df_features['gender_institution'] = df_features['gender'] + '_' + df_features['institution_type']
    
    # Parent education combined score
    edu_score_map = {'No Education': 0, 'Primary': 1, 'Secondary': 2, 'Higher Secondary': 3, 'University': 4}
    df_features['parent_education_score'] = (
        df_features['mother_education'].map(edu_score_map) + 
        df_features['father_education'].map(edu_score_map)
    )
    
    # Risk indicators
    df_features['high_absenteeism'] = (df_features['days_absent_last_term'] > 15).astype(int)
    df_features['low_previous_performance'] = (df_features['previous_year_gpa'] < 2.5).astype(int)
    df_features['limited_resources'] = (
        (df_features['home_internet'] == 0) & 
        (df_features['socioeconomic_status'] == 'Low')
    ).astype(int)
    
    return df_features

# Apply feature engineering
modeling_features = create_features(modeling_data)

# Prepare features for modeling
def prepare_features(df, target_col):
    """Prepare features for machine learning."""
    # Separate numerical and categorical features
    numerical_features = [
        'age', 'grade_level', 'previous_year_gpa', 'days_absent_last_term',
        'extracurricular_activities', 'home_internet', 'study_hours_per_day',
        'teacher_student_ratio', 'school_infrastructure_score',
        'age_grade_ratio', 'gpa_improvement_potential', 'attendance_rate',
        'study_intensity', 'parent_education_score', 'high_absenteeism',
        'low_previous_performance', 'limited_resources'
    ]
    
    categorical_features = [
        'division', 'gender', 'institution_type', 'socioeconomic_status',
        'area_type', 'mother_education', 'father_education', 'family_income',
        'ses_area', 'gender_institution'
    ]
    
    # Create feature matrix
    X_numerical = df[numerical_features]
    
    # One-hot encode categorical features
    X_categorical = pd.get_dummies(df[categorical_features], drop_first=True)
    
    # Combine features
    X = pd.concat([X_numerical, X_categorical], axis=1)
    y = df[target_col]
    
    return X, y, numerical_features, list(X_categorical.columns)

# Prepare features for GPA prediction
X, y, num_features, cat_features = prepare_features(modeling_features, 'current_gpa')

print(f"Feature matrix shape: {X.shape}")
print(f"Number of numerical features: {len(num_features)}")
print(f"Number of categorical features: {len(cat_features)}")
print(f"\nFeature names:")
print(f"Numerical: {num_features}")
print(f"Categorical (first 10): {cat_features[:10]}")

# Check for missing values
print(f"\nMissing values: {X.isnull().sum().sum()}")
print(f"Target variable range: {y.min():.2f} to {y.max():.2f}")

## 3. Model Development and Training

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Scale only numerical features
X_train_scaled[num_features] = scaler.fit_transform(X_train[num_features])
X_test_scaled[num_features] = scaler.transform(X_test[num_features])

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

# Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=0.1),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf'),
    'Neural Network': MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
}

# Train and evaluate models
model_results = {}
trained_models = {}

for name, model in models.items():
    print(f"Training {name}...")
    
    # Use scaled data for models that benefit from scaling
    if name in ['SVR', 'Neural Network', 'Ridge Regression', 'Lasso Regression']:
        X_train_model = X_train_scaled
        X_test_model = X_test_scaled
    else:
        X_train_model = X_train
        X_test_model = X_test
    
    # Train model
    model.fit(X_train_model, y_train)
    
    # Make predictions
    y_pred_train = model.predict(X_train_model)
    y_pred_test = model.predict(X_test_model)
    
    # Calculate metrics
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    
    # Cross-validation score
    cv_scores = cross_val_score(model, X_train_model, y_train, cv=5, scoring='r2')
    
    model_results[name] = {
        'train_rmse': train_rmse,
        'test_rmse': test_rmse,
        'train_r2': train_r2,
        'test_r2': test_r2,
        'train_mae': train_mae,
        'test_mae': test_mae,
        'cv_r2_mean': cv_scores.mean(),
        'cv_r2_std': cv_scores.std(),
        'predictions': y_pred_test
    }
    
    trained_models[name] = model

print("\nModel training completed!")

# Create results summary
results_df = pd.DataFrame(model_results).T
results_df = results_df.drop('predictions', axis=1)
results_df = results_df.round(4)

print("Model Performance Summary:")
print(results_df.sort_values('test_r2', ascending=False))

## 4. Model Evaluation and Comparison

In [ ]:
# Visualization of model performance
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# R² scores comparison
model_names = list(model_results.keys())
train_r2_scores = [model_results[name]['train_r2'] for name in model_names]
test_r2_scores = [model_results[name]['test_r2'] for name in model_names]

x = np.arange(len(model_names))
width = 0.35

ax1.bar(x - width/2, train_r2_scores, width, label='Train R²', alpha=0.8)
ax1.bar(x + width/2, test_r2_scores, width, label='Test R²', alpha=0.8)
ax1.set_xlabel('Models')
ax1.set_ylabel('R² Score')
ax1.set_title('Model Performance Comparison (R²)')
ax1.set_xticks(x)
ax1.set_xticklabels(model_names, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# RMSE comparison
train_rmse_scores = [model_results[name]['train_rmse'] for name in model_names]
test_rmse_scores = [model_results[name]['test_rmse'] for name in model_names]

ax2.bar(x - width/2, train_rmse_scores, width, label='Train RMSE', alpha=0.8)
ax2.bar(x + width/2, test_rmse_scores, width, label='Test RMSE', alpha=0.8)
ax2.set_xlabel('Models')
ax2.set_ylabel('RMSE')
ax2.set_title('Model Performance Comparison (RMSE)')
ax2.set_xticks(x)
ax2.set_xticklabels(model_names, rotation=45, ha='right')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Cross-validation scores
cv_means = [model_results[name]['cv_r2_mean'] for name in model_names]
cv_stds = [model_results[name]['cv_r2_std'] for name in model_names]

ax3.bar(model_names, cv_means, yerr=cv_stds, capsize=5, alpha=0.8)
ax3.set_xlabel('Models')
ax3.set_ylabel('Cross-Validation R²')
ax3.set_title('Cross-Validation Performance')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(True, alpha=0.3)

# Prediction vs Actual scatter plot for best model
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['test_r2'])
best_predictions = model_results[best_model_name]['predictions']

ax4.scatter(y_test, best_predictions, alpha=0.6)
ax4.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax4.set_xlabel('Actual GPA')
ax4.set_ylabel('Predicted GPA')
ax4.set_title(f'Best Model: {best_model_name}\n(Test R² = {model_results[best_model_name]["test_r2"]:.3f})')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Model ranking
print(f"\nBest performing model: {best_model_name}")
print(f"Test R²: {model_results[best_model_name]['test_r2']:.4f}")
print(f"Test RMSE: {model_results[best_model_name]['test_rmse']:.4f}")
print(f"Test MAE: {model_results[best_model_name]['test_mae']:.4f}")

# Calculate prediction intervals
residuals = y_test - best_predictions
residual_std = np.std(residuals)

print(f"\nPrediction Accuracy Analysis:")
print(f"Residual standard deviation: {residual_std:.3f}")
print(f"95% prediction interval: ±{1.96 * residual_std:.3f} GPA points")

# Accuracy within thresholds
within_01 = np.mean(np.abs(residuals) <= 0.1) * 100
within_02 = np.mean(np.abs(residuals) <= 0.2) * 100
within_03 = np.mean(np.abs(residuals) <= 0.3) * 100

print(f"Predictions within 0.1 GPA points: {within_01:.1f}%")
print(f"Predictions within 0.2 GPA points: {within_02:.1f}%")
print(f"Predictions within 0.3 GPA points: {within_03:.1f}%")

## 5. Feature Importance Analysis

In [ ]:
# Feature importance analysis for tree-based models
def analyze_feature_importance(model, model_name, feature_names, X_test, y_test):
    """Analyze feature importance using multiple methods."""
    importance_data = {}
    
    # Built-in feature importance (for tree-based models)
    if hasattr(model, 'feature_importances_'):
        importance_data['built_in'] = model.feature_importances_
    
    # Permutation importance
    perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
    importance_data['permutation'] = perm_importance.importances_mean
    
    return importance_data

# Analyze feature importance for the best models
top_models = ['Random Forest', 'Gradient Boosting']

feature_importance_results = {}

for model_name in top_models:
    if model_name in trained_models:
        model = trained_models[model_name]
        
        # Use appropriate test data
        if model_name in ['SVR', 'Neural Network', 'Ridge Regression', 'Lasso Regression']:
            X_test_model = X_test_scaled
        else:
            X_test_model = X_test
        
        importance_data = analyze_feature_importance(model, model_name, X.columns, X_test_model, y_test)
        feature_importance_results[model_name] = importance_data

# Visualize feature importance
fig, axes = plt.subplots(len(top_models), 2, figsize=(15, 6*len(top_models)))
if len(top_models) == 1:
    axes = [axes]

for i, model_name in enumerate(top_models):
    if model_name in feature_importance_results:
        importance_data = feature_importance_results[model_name]
        
        # Built-in importance
        if 'built_in' in importance_data:
            built_in_importance = importance_data['built_in']
            feature_importance_df = pd.DataFrame({
                'feature': X.columns,
                'importance': built_in_importance
            }).sort_values('importance', ascending=True).tail(15)
            
            axes[i][0].barh(feature_importance_df['feature'], feature_importance_df['importance'])
            axes[i][0].set_title(f'{model_name} - Built-in Feature Importance')
            axes[i][0].set_xlabel('Importance')
        
        # Permutation importance
        perm_importance = importance_data['permutation']
        perm_importance_df = pd.DataFrame({
            'feature': X.columns,
            'importance': perm_importance
        }).sort_values('importance', ascending=True).tail(15)
        
        axes[i][1].barh(perm_importance_df['feature'], perm_importance_df['importance'])
        axes[i][1].set_title(f'{model_name} - Permutation Importance')
        axes[i][1].set_xlabel('Importance')

plt.tight_layout()
plt.show()

# Top features summary
print("TOP FEATURES ANALYSIS")
print("=" * 50)

for model_name in top_models:
    if model_name in feature_importance_results:
        print(f"\n{model_name}:")
        
        # Permutation importance (more reliable)
        perm_importance = feature_importance_results[model_name]['permutation']
        top_features = pd.DataFrame({
            'feature': X.columns,
            'importance': perm_importance
        }).sort_values('importance', ascending=False).head(10)
        
        print("Top 10 features (Permutation Importance):")
        for idx, row in top_features.iterrows():
            print(f"  {row['feature']}: {row['importance']:.4f}")

## 6. Dropout Risk Prediction Model

In [ ]:
# Develop dropout risk prediction model
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# Prepare data for dropout prediction
X_dropout, y_dropout, _, _ = prepare_features(modeling_features, 'dropout_risk')

# Split data
X_train_dropout, X_test_dropout, y_train_dropout, y_test_dropout = train_test_split(
    X_dropout, y_dropout, test_size=0.2, random_state=42, stratify=y_dropout
)

# Scale features
X_train_dropout_scaled = X_train_dropout.copy()
X_test_dropout_scaled = X_test_dropout.copy()
X_train_dropout_scaled[num_features] = scaler.fit_transform(X_train_dropout[num_features])
X_test_dropout_scaled[num_features] = scaler.transform(X_test_dropout[num_features])

# Define classification models
dropout_models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# Train and evaluate dropout models
dropout_results = {}

for name, model in dropout_models.items():
    print(f"Training {name} for dropout prediction...")
    
    # Use scaled data for logistic regression
    if name == 'Logistic Regression':
        X_train_model = X_train_dropout_scaled
        X_test_model = X_test_dropout_scaled
    else:
        X_train_model = X_train_dropout
        X_test_model = X_test_dropout
    
    # Train model
    model.fit(X_train_model, y_train_dropout)
    
    # Make predictions
    y_pred_dropout = model.predict(X_test_model)
    y_pred_proba_dropout = model.predict_proba(X_test_model)[:, 1]
    
    # Calculate metrics
    auc_score = roc_auc_score(y_test_dropout, y_pred_proba_dropout)
    
    dropout_results[name] = {
        'model': model,
        'predictions': y_pred_dropout,
        'probabilities': y_pred_proba_dropout,
        'auc_score': auc_score
    }

# Find best dropout model
best_dropout_model = max(dropout_results.keys(), key=lambda x: dropout_results[x]['auc_score'])

print(f"\nBest dropout prediction model: {best_dropout_model}")
print(f"AUC Score: {dropout_results[best_dropout_model]['auc_score']:.4f}")

# Detailed evaluation of best model
best_model = dropout_results[best_dropout_model]['model']
best_predictions = dropout_results[best_dropout_model]['predictions']
best_probabilities = dropout_results[best_dropout_model]['probabilities']

print(f"\nClassification Report for {best_dropout_model}:")
print(classification_report(y_test_dropout, best_predictions))

# Confusion matrix
cm = confusion_matrix(y_test_dropout, best_predictions)

# Visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# AUC comparison
model_names_dropout = list(dropout_results.keys())
auc_scores = [dropout_results[name]['auc_score'] for name in model_names_dropout]

ax1.bar(model_names_dropout, auc_scores, color='skyblue', alpha=0.8)
ax1.set_xlabel('Models')
ax1.set_ylabel('AUC Score')
ax1.set_title('Dropout Prediction Model Comparison')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3)

# Confusion matrix heatmap
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2)
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Actual')
ax2.set_title(f'Confusion Matrix - {best_dropout_model}')

# ROC curve
fpr, tpr, _ = roc_curve(y_test_dropout, best_probabilities)
ax3.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {dropout_results[best_dropout_model]["auc_score"]:.3f})')
ax3.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate')
ax3.set_title('ROC Curve')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Dropout probability distribution
ax4.hist(best_probabilities[y_test_dropout == 0], bins=30, alpha=0.7, label='Not at Risk', density=True)
ax4.hist(best_probabilities[y_test_dropout == 1], bins=30, alpha=0.7, label='At Risk', density=True)
ax4.set_xlabel('Dropout Probability')
ax4.set_ylabel('Density')
ax4.set_title('Dropout Probability Distribution')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Model Ensemble and Final Predictions

In [ ]:
# Create ensemble model for GPA prediction
ensemble_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('ridge', Ridge(alpha=1.0))
]

ensemble_regressor = VotingRegressor(ensemble_models)
ensemble_regressor.fit(X_train, y_train)

# Ensemble predictions
ensemble_predictions = ensemble_regressor.predict(X_test)
ensemble_r2 = r2_score(y_test, ensemble_predictions)
ensemble_rmse = np.sqrt(mean_squared_error(y_test, ensemble_predictions))

print(f"Ensemble Model Performance:")
print(f"R² Score: {ensemble_r2:.4f}")
print(f"RMSE: {ensemble_rmse:.4f}")

# Compare with best individual model
best_individual_r2 = model_results[best_model_name]['test_r2']
improvement = ensemble_r2 - best_individual_r2

print(f"\nComparison with best individual model ({best_model_name}):")
print(f"Individual model R²: {best_individual_r2:.4f}")
print(f"Ensemble model R²: {ensemble_r2:.4f}")
print(f"Improvement: {improvement:+.4f}")

# Create prediction function
def predict_student_performance(student_features, gpa_model, dropout_model, scaler, feature_columns):
    """Predict student GPA and dropout risk."""
    # Ensure features are in correct order
    student_df = pd.DataFrame([student_features])
    
    # Apply feature engineering
    student_engineered = create_features(student_df)
    
    # Prepare features
    X_student, _, _, _ = prepare_features(student_engineered, 'current_gpa')
    
    # Ensure all columns are present
    for col in feature_columns:
        if col not in X_student.columns:
            X_student[col] = 0
    
    X_student = X_student[feature_columns]
    
    # Scale numerical features
    X_student_scaled = X_student.copy()
    X_student_scaled[num_features] = scaler.transform(X_student[num_features])
    
    # Make predictions
    gpa_prediction = gpa_model.predict(X_student)[0]
    dropout_risk = dropout_model.predict_proba(X_student_scaled)[:, 1][0]
    
    return gpa_prediction, dropout_risk

# Example prediction
example_student = {
    'student_id': 'S_NEW_001',
    'division': 'Dhaka',
    'gender': 'Female',
    'age': 16,
    'grade_level': 10,
    'institution_type': 'Government',
    'socioeconomic_status': 'Medium',
    'area_type': 'Urban',
    'mother_education': 'Secondary',
    'father_education': 'Higher Secondary',
    'family_income': 'Medium',
    'previous_year_gpa': 3.5,
    'days_absent_last_term': 8,
    'extracurricular_activities': 2,
    'home_internet': 1,
    'study_hours_per_day': 4.5,
    'teacher_student_ratio': 35,
    'school_infrastructure_score': 7.5
}

# Get best models
best_gpa_model = ensemble_regressor
best_dropout_model_obj = dropout_results[best_dropout_model]['model']

# Make prediction
predicted_gpa, predicted_dropout_risk = predict_student_performance(
    example_student, best_gpa_model, best_dropout_model_obj, scaler, X.columns
)

print(f"\nExample Student Prediction:")
print(f"Student Profile: {example_student['gender']}, {example_student['age']} years old, Grade {example_student['grade_level']}")
print(f"Previous GPA: {example_student['previous_year_gpa']}")
print(f"\nPredictions:")
print(f"Predicted GPA: {predicted_gpa:.2f}")
print(f"Dropout Risk: {predicted_dropout_risk:.1%}")

# Risk categorization
if predicted_dropout_risk < 0.1:
    risk_level = 'Low'
elif predicted_dropout_risk < 0.3:
    risk_level = 'Medium'
else:
    risk_level = 'High'

print(f"Risk Level: {risk_level}")

# Save models
model_save_path = Path('../../models')
model_save_path.mkdir(exist_ok=True)

joblib.dump(best_gpa_model, model_save_path / 'gpa_prediction_model.pkl')
joblib.dump(best_dropout_model_obj, model_save_path / 'dropout_prediction_model.pkl')
joblib.dump(scaler, model_save_path / 'feature_scaler.pkl')

print(f"\nModels saved to {model_save_path}:")

## 8. Model Insights and Recommendations

In [ ]:
# Generate comprehensive insights
print("🎯 PREDICTIVE MODELING INSIGHTS")
print("=" * 50)

# Model performance summary
print(f"📊 Model Performance Summary:")
print(f"   • Best GPA prediction model: Ensemble (R² = {ensemble_r2:.3f})")
print(f"   • Best dropout prediction model: {best_dropout_model} (AUC = {dropout_results[best_dropout_model]['auc_score']:.3f})")
print(f"   • Prediction accuracy within 0.2 GPA: {within_02:.1f}%")

# Feature insights
print(f"🔍 Key Predictive Factors:")

if 'Random Forest' in feature_importance_results:
    rf_importance = feature_importance_results['Random Forest']['permutation']
    top_predictors = pd.DataFrame({
        'feature': X.columns,
        'importance': rf_importance
    }).sort_values('importance', ascending=False).head(5)
    
    print("   Top 5 predictors of academic performance:")
    for idx, row in top_predictors.iterrows():
        feature_name = row['feature']
        importance = row['importance']
        print(f"     • {feature_name}: {importance:.4f}")

# Risk factor analysis
print(f"⚠️ Dropout Risk Factors:")
high_risk_students = modeling_features[modeling_features['dropout_risk'] == 1]
low_risk_students = modeling_features[modeling_features['dropout_risk'] == 0]

# Compare characteristics
risk_comparison = {
    'Average GPA': {
        'High Risk': high_risk_students['current_gpa'].mean(),
        'Low Risk': low_risk_students['current_gpa'].mean()
    },
    'Average Absences': {
        'High Risk': high_risk_students['days_absent_last_term'].mean(),
        'Low Risk': low_risk_students['days_absent_last_term'].mean()
    },
    'Low SES %': {
        'High Risk': (high_risk_students['socioeconomic_status'] == 'Low').mean() * 100,
        'Low Risk': (low_risk_students['socioeconomic_status'] == 'Low').mean() * 100
    }
}

for metric, values in risk_comparison.items():
    print(f"   • {metric}:")
    print(f"     - High risk students: {values['High Risk']:.2f}")
    print(f"     - Low risk students: {values['Low Risk']:.2f}")

# Intervention recommendations
print(f"💡 Evidence-Based Intervention Recommendations:")
print(f"
🎯 Early Warning System:")
print(f"   • Monitor students with GPA < 2.5 (high dropout risk)")
print(f"   • Track attendance closely (>10 absences = increased risk)")
print(f"   • Identify students with multiple risk factors")

print(f"🏋️ Targeted Interventions:")
print(f"   • Academic support for students with previous GPA < 3.0")
print(f"   • Attendance improvement programs")
print(f"   • Family engagement initiatives for low SES students")
print(f"   • Technology access programs (home internet/devices)")

print(f"📊 Performance Optimization:")
print(f"   • Encourage study time of 4-6 hours per day")
print(f"   • Promote extracurricular participation")
print(f"   • Improve school infrastructure where needed")
print(f"   • Reduce teacher-student ratios in underperforming areas")

print(f"📝 Monitoring and Evaluation:")
print(f"   • Implement quarterly prediction updates")
print(f"   • Track intervention effectiveness")
print(f"   • Adjust models based on new data")
print(f"   • Validate predictions against actual outcomes")

# Model limitations and future improvements
print(f"🔄 Model Limitations and Future Improvements:")
print(f"   • Current model explains {ensemble_r2:.1%} of GPA variance")
print(f"   • Consider adding psychological and social factors")
print(f"   • Incorporate longitudinal tracking data")
print(f"   • Add teacher quality and classroom environment variables")
print(f"   • Develop subject-specific prediction models")